In [1]:
import os
os.environ['HF_ENDPOINT']='https://hf-mirror.com'
os.environ['CUDA_VISIBLE_DEVICES']='1'
import torch
import datasets
import transformers
from transformers import AutoModelForQuestionAnswering,AutoModelForCausalLM
from nn_pruning.patch_coordinator import ModelPatchingCoordinator
from nn_pruning.model_patcher import ModelPatcher
import torch
from nn_pruning.patch_coordinator import SparseTrainingArguments

datasets.logging.set_verbosity_error()
transformers.logging.set_verbosity_error()
print(f"Using transformers v{transformers.__version__} and datasets v{datasets.__version__} and torch v{torch.__version__}")

/home/cmyu/anaconda3/envs/tvm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using transformers v4.40.0 and datasets v3.0.1 and torch v2.0.1+cu117


In [11]:
from examples.question_answering.qa_qwen2 import Qwen2ForQuestionAnswering

In [16]:
model_name = 'Qwen/Qwen2-1.5B'
model = Qwen2ForQuestionAnswering.from_pretrained(model_name)

/home/cmyu/anaconda3/envs/tvm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
type(model.config)

transformers.models.qwen2.configuration_qwen2.Qwen2Config

In [4]:
sparse_args = SparseTrainingArguments()

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


mpc = ModelPatchingCoordinator(
    sparse_args=sparse_args, 
    device=device, 
    cache_dir="checkpoints", 
    logit_names="logits", 
    model_name_or_path=model_name,
    teacher_constructor=None)

# 对模型进行补丁
patched_model = mpc.patch_model(model)

In [14]:
for n,m in model.named_modules():
    print(n)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.

In [15]:
model.model.embed_tokens.weight

Parameter containing:
tensor([[ 0.0046, -0.0153,  0.0017,  ...,  0.0133,  0.0251,  0.0131],
        [ 0.0238, -0.0071,  0.0131,  ...,  0.0206, -0.0037,  0.0133],
        [-0.0159, -0.0078,  0.0012,  ...,  0.0051, -0.0183,  0.0079],
        ...,
        [-0.0136,  0.0193, -0.0072,  ..., -0.0040,  0.0003, -0.0034],
        [-0.0136,  0.0193, -0.0072,  ..., -0.0040,  0.0003, -0.0034],
        [-0.0136,  0.0193, -0.0072,  ..., -0.0040,  0.0003, -0.0034]],
       requires_grad=True)

In [9]:
AutoModelForQuestionAnswering.from_pretrained('gpt2')

/home/cmyu/anaconda3/envs/tvm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT2ForQuestionAnswering(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (qa_outputs): Linear(in_features=768, out_features=2, bias=True)
)

: 